In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Infos zum Datensatz

In [3]:
chunksize = 1_000_000
reader = pd.read_csv("data/2023_Yellow_Taxi_Trip_Data.csv", chunksize=chunksize)

# Den ersten Chunk holen
first_chunk = next(reader)

# Info anzeigen
print(first_chunk.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1000000 non-null  int64  
 1   tpep_pickup_datetime   1000000 non-null  object 
 2   tpep_dropoff_datetime  1000000 non-null  object 
 3   passenger_count        1000000 non-null  int64  
 4   trip_distance          1000000 non-null  float64
 5   RatecodeID             1000000 non-null  int64  
 6   store_and_fwd_flag     1000000 non-null  object 
 7   PULocationID           1000000 non-null  int64  
 8   DOLocationID           1000000 non-null  int64  
 9   payment_type           1000000 non-null  int64  
 10  fare_amount            1000000 non-null  float64
 11  extra                  1000000 non-null  float64
 12  mta_tax                1000000 non-null  float64
 13  tip_amount             1000000 non-null  float64
 14  tolls_amount       

Wie wir sehen können ist der Datensatz recht gross und braucht viel Speicher, weshalb es sich nur in chunks bearbeiten lässt. 

Im Folgenden wird es vorallem darum gehen den Datensatz zu optimieren und wichtige Kennzahlen zu berechnen.

### Beschreiben des Datensatzes

In [4]:
second_chunk = next(reader)

print(second_chunk.describe())

             VendorID  passenger_count   trip_distance      RatecodeID  \
count  1000000.000000   1000000.000000  1000000.000000  1000000.000000   
mean         1.732321         1.364828        3.372839        1.435825   
std          0.442750         0.896679       24.952718        6.013780   
min          1.000000         0.000000        0.000000        1.000000   
25%          1.000000         1.000000        1.060000        1.000000   
50%          2.000000         1.000000        1.760000        1.000000   
75%          2.000000         1.000000        3.250000        1.000000   
max          2.000000         9.000000     9683.760000       99.000000   

         PULocationID    DOLocationID    payment_type     fare_amount  \
count  1000000.000000  1000000.000000  1000000.000000  1000000.000000   
mean       166.420072      164.483686        1.215696       18.193923   
std         64.097900       69.799382        0.496670       17.590646   
min          1.000000        1.000000    

In [5]:
df_header = pd.read_csv("data/2023_Yellow_Taxi_Trip_Data.csv", nrows=0)
print("Der Datensatz enthält folgende Informationen")
original_header = df_header.columns.tolist()
print("Spaltennamen:\n", original_header)
print("\nAnzahl Header: ", len(original_header))

with open("data/2023_Yellow_Taxi_Trip_Data.csv", "r", encoding="utf-8") as f:
    total_lines = sum(1 for _ in f) - 1

print(f"\nDie Datei hat ca. {total_lines:,} Zeilen.")

Der Datensatz enthält folgende Informationen
Spaltennamen:
 ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee']

Anzahl Header:  19

Die Datei hat ca. 38,310,226 Zeilen.


Um die Anzahl Zeilen zu erhalten konnte ich keine pandas einsetzen, da jedesmal das kernel kaputt gegangen ist. 
Unser Datensatz hat also um die 38 Millionen Zeilen/Taxifahrten im Jahr 2023 und zu einer Fahrt sind viel Daten vorhanden, wie zum Beispiel Preis, Uhrzeit und Distanz.

### Mögliche Datenverbesserung

Dank der Info- und Describe-Funktionen können wir uns nun den gesamten Datensatz ansehen und erkennen, welche Spalten für unsere Analyse nicht nützlich sind. Diese Spalten werden gelöscht, um Arbeitsspeicher (RAM) zu sparen und die Datengröße zu reduzieren.

- VendorID: Hat nur die Werte 1, 2 oder 6. Da wir diese Information nicht mit etwas Nützlichem verknüpfen können, kann die Spalte
            gelöscht werden.

- RatecodeID: Gleiches wie zuvor – die Werte sind meist 1 oder 99, in den meisten Fällen jedoch 1. Daher ist sie nicht hilfreich.

- store_and_fwd_flag: Immer derselbe Wert und die Bedeutung ist unklar. Diese Spalte kann ebenfalls entfernt werden.

- mta_tax: Hat immer denselben Wert (0.5) – es gibt zwar einige fehlerhafte Einträge, aber grundsätzlich keine Variation, daher kann
           sie gelöscht werden.

### Überprüfen der Spaltenlöschung

In [6]:
print("Datensatz nach Löschung:\n")
df = pd.read_csv("data/Taxi_Data_improved.csv", nrows=0)
print(f"Datensatz hat jetzt {df.shape[1]} Spalten")

Datensatz nach Löschung:

Datensatz hat jetzt 15 Spalten


### Erstellen eines repräsentativen Datensatzes

Unser Datensatz hat aktuell mehr als 38 Millionen Taxifahrten, es ist unmöglich (auch mit pandas) diesen in einem Stück einzulesen und zu analysieren, desshalb haben wir uns entschieden einen kleineren repräsentativen Datensatz zu erstellen. Zuerst wird das original chunkweise geshuffelt und anschliessen aus jedem gemischten chunk eine Prozentsatz an Daten random herausgenommen und in einem neuen Datensatz gespeichert. Der neue Datensatz sollte 1 Millionen Taxifahrten beinhalten.

In [8]:
print(f"Daten des originalen Datensatzes: {total_lines:,}")

with open("data/Taxi_sample_1M.csv", "r", encoding="utf-8") as f:
    new_total_lines = sum(1 for _ in f) - 1
print(f"Daten des neuen Datensatzes: {new_total_lines:,}")

Daten des originalen Datensatzes: 38,310,226
Daten des neuen Datensatzes: 999,997


### Wechsel zu Datetime

Aktuell liegt die Abfahrts und Ankunfts Zeit in der US-Datums form vor. Um die Zeiten für Rechnungen benutzen zu können, müssen wir die Zeitangaben in Datetime vorleigen haben. deshalb wurde das pythonscript change_to_datetime erstellt welches diese Angaben umwandelt

In [9]:
example_1 = pd.read_csv("data/2023_Yellow_Taxi_Trip_Data.csv", nrows=3)
print("Zeitangabe vor Änderung:\n")
print(example_1.iloc[:, 1:3])

example_2 = pd.read_csv("data/Taxi_final_1M.csv", nrows=3)
print("\nZeitangabe nach Änderung:\n")
print(example_2.iloc[:, :2])

Zeitangabe vor Änderung:

     tpep_pickup_datetime   tpep_dropoff_datetime
0  01/01/2023 12:32:10 AM  01/01/2023 12:40:36 AM
1  01/01/2023 12:55:08 AM  01/01/2023 01:01:27 AM
2  01/01/2023 12:25:04 AM  01/01/2023 12:37:49 AM

Zeitangabe nach Änderung:

  tpep_pickup_datetime tpep_dropoff_datetime
0  2023-02-06 18:31:28   2023-02-06 18:41:28
1  2023-01-13 12:22:41   2023-01-13 12:54:42
2  2023-01-24 12:53:51   2023-01-24 13:06:09


### Generieren von neuen Spalten

Neben dem Wechsel zu Datetime, wird mit dem gleichen Pythonscript die Taxifahrtdauer und die durchschnitts Geschwindigkeit berechnet und hinten als neue Spalten eingefühgt. Zudem wird die Spalte trip_distance in Kilometer umgerechnet.

In [18]:
example_3 = pd.read_csv("data/Taxi_final_1M.csv", nrows=6)
print("\nNeu eingefühgte Spalten:\n")
print(example_3.iloc[:,15:])


Neu eingefühgte Spalten:

   trip_duration  average_speed
0           10.0           27.4
1           32.0            0.0
2           12.3           18.1
3            4.4           24.9
4           11.4           24.9
5            5.7           21.1


### Wichtige Kennzahlen für Fahrzeit, Distanz und Gesamtpreis

(Mittelwert, Median, Modus, Quartile)

In [22]:
data = pd.read_csv("data/Taxi_final_1M.csv")

print("Werte für Fahrzeit:\n")
print(data['trip_duration'].describe())
print(f"Median: {data['trip_duration'].median()}")
print(f"Modus: {data['trip_duration'].mode()}")

print("Werte für Distanz:\n")
print(data['trip_distance'].describe())
print(f"Median: {data['trip_distance'].median()}")
print(f"Modus: {data['trip_distance'].mode()}")

print("Werte für Gesamtpreis:\n")
print(data['total_amount'].describe())
print(f"Median: {data['total_amount'].median()}")
print(f"Modus: {data['total_amount'].mode()}")

Werte für Fahrzeit:

count    999997.000000
mean         17.447879
std          40.936206
min         -52.200000
25%           7.600000
50%          12.600000
75%          20.700000
max        3713.800000
Name: trip_duration, dtype: float64
Median: 12.6
Modus: 0    7.0
Name: trip_duration, dtype: float64
Werte für Distanz:

count    999997.000000
mean          6.379764
std         218.072658
min           0.000000
25%           1.700000
50%           2.900000
75%           5.500000
max      148560.100000
Name: trip_distance, dtype: float64
Median: 2.9
Modus: 0    1.4
Name: trip_distance, dtype: float64
Werte für Gesamtpreis:

count    999997.000000
mean         28.386058
std          23.687311
min        -901.000000
25%          15.950000
50%          21.000000
75%          30.720000
max        2451.000000
Name: total_amount, dtype: float64
Median: 21.0
Modus: 0    16.8
Name: total_amount, dtype: float64
